# Capstone Week 3 Project - Part 3

## Segmenting and Clustering Neighborhoods in Toronto

#### To set the context, the below is a continuation of parts 1 and 2. Scroll to the bottom to pick with part 3

Get neighborhood data for Toronto

In [1]:
# Import all needed packages here
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml
from pprint import pprint
from collections import defaultdict
import csv

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# go get the web page
web_page = requests.get(url)
# make some tasty soup
soup = BeautifulSoup(web_page.content, 'html.parser')

I viewed the Wikipedia page in Firefox then pressed Ctrl+U to see the source. The data is in a table of class 'wikitable'. With this I will:
1. Get the HTML for that specific table
2. Parse out the row data
3. Load the data into a Python list
4. Create the pandas DataFrame from the list

**Note:** I am using ``defaultdict`` from the handy Python ``collections`` module to accumulate a list of neighborhoods in a borough. Then I do some data wrangling to transform that list into a comma separated string and output the results to the list for pandas DataFrame creation.

#### DataFrame Creation Below...

In [107]:
trono_fsas = soup.find('table', class_='wikitable')
html_rows = trono_fsas.find_all('tr')
data = []
d_pc_boro = defaultdict(list)
# loop thru the rows, populate dict with key (boro, pc), append list of neighborhoods
for tr in html_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if len(row) == 3:
        pc = row[0]
        boro = row[1]
        nbh = row[2]
        if boro == 'Not assigned':  # ignore cells with unassigned Borough
            continue
        if nbh == 'Not assigned':  # name unassigned neighborhoods to the name of the Borough
            nbh = boro
        d_pc_boro[(boro, pc)].append(nbh)  # group the neighborhoods
# loop thru the dict, turn list of neighborhoods to a string and output a list for pandas
for (bro, pcode), hoods in sorted(d_pc_boro.items()):
    nbstr = ', '.join(hoods)  # concatenate the hoods
    data.append([pcode, bro, nbstr])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M4P,Central Toronto,Davisville North
2,M4R,Central Toronto,North Toronto West
3,M4S,Central Toronto,Davisville
4,M4T,Central Toronto,"Moore Park, Summerhill East"


The above output already gives the shape but the assignment says to use the shape method, so here it is...

In [22]:
df.shape

(103, 3)

---

### Part 2 starts here. Make sure the above cells are run...
Download the Geospatial data

In [5]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


Look at the data...

In [23]:
with open('Geospatial_Coordinates.csv') as csv_data:
    longlatdat = csv_data.readlines()
    # longlatdat

In [24]:
longlatdat[:5]

['Postal Code,Latitude,Longitude\n',
 'M1B,43.8066863,-79.1943534\n',
 'M1C,43.7845351,-79.1604971\n',
 'M1E,43.7635726,-79.1887115\n',
 'M1G,43.7709921,-79.2169174\n']

There are three columns, so create a dictionary for each postal code so we can look them when we loop through our data.

In [8]:
d_postal_code = {}  # {'code': (lat, long)}
with open('Geospatial_Coordinates.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        d_postal_code[row['Postal Code']] = (row['Latitude'], row['Longitude'])

In [9]:
data[:5]

[['M4N', 'Central Toronto', 'Lawrence Park'],
 ['M4P', 'Central Toronto', 'Davisville North'],
 ['M4R', 'Central Toronto', 'North Toronto West'],
 ['M4S', 'Central Toronto', 'Davisville'],
 ['M4T', 'Central Toronto', 'Moore Park, Summerhill East']]

**Critical Note:** Make sure the data appears in the above. If not rerun the **DataFrame creation** cell above!!

Loop thru the original data and create a list with the coordinate data.

In [25]:
l_hoods = []
for pc, boro, hoods in data:
    lat, lng = d_postal_code[pc]
    l_hoods.append([pc, boro, hoods, lat, lng])
l_hoods[:5]

[['M4N', 'Central Toronto', 'Lawrence Park', '43.7280205', '-79.3887901'],
 ['M4P', 'Central Toronto', 'Davisville North', '43.7127511', '-79.3901975'],
 ['M4R', 'Central Toronto', 'North Toronto West', '43.7153834', '-79.4056784'],
 ['M4S', 'Central Toronto', 'Davisville', '43.7043244', '-79.3887901'],
 ['M4T',
  'Central Toronto',
  'Moore Park, Summerhill East',
  '43.6895743',
  '-79.3831599']]

Create the new DataFrame with the coordinate data...

In [108]:
coord_df = pd.DataFrame(l_hoods, columns=['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])
coord_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901
1,M4P,Central Toronto,Davisville North,43.7127511,-79.3901975
2,M4R,Central Toronto,North Toronto West,43.7153834,-79.4056784
3,M4S,Central Toronto,Davisville,43.7043244,-79.3887901
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.6895743,-79.3831599


In [27]:
coord_df.shape

(103, 5)

---

### Part 3 starts here. Make sure the above cells are run...

``click Cell > Run All``

1. Replicate the NYC analysis
2. Generate maps
3. Evaluate Clusterfication



In [30]:
# Import all needed packages here
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml
from pprint import pprint
from collections import defaultdict
import csv
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
from geopy.geocoders import Nominatim
import folium
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Explore the Dataset

Create a list of Toronto neighborhoods.

In [49]:
trono_hoods = []
for pc, boro, hoods, lat, lng in l_hoods:
    idx = boro.find('Toronto')
    if idx > -1:
        #trono_hoods.append([pc, boro, hoods, lat, lng])
        trono_hoods.append([boro, hoods, float(lat), float(lng)])

trono_hoods.sort()
len(trono_hoods)

38

Create the DataFrame of Trono Neighborhoods

In [50]:
trono_df = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude'])
for boro, hood, lat, lng in trono_hoods:
    trono_df = trono_df.append({'Borough': boro,
                                'Neighborhood': hood,
                                'Latitude': lat,
                                'Longitude': lng}, ignore_index=True)

trono_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,Davisville,43.704324,-79.388790
1,Central Toronto,Davisville North,43.712751,-79.390197
2,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
3,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
4,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Get Coordinates for Toronto

In [33]:
cityname = 'Toronto, Canada'
geoloc8r = Nominatim()
location = geoloc8r.geocode(cityname)
latitude = location.latitude
lngitude = location.longitude
print('Coordinates for Trono: {}, {}.'.format(latitude, lngitude))

/home/steve.orr/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Coordinates for Trono: 43.653963, -79.387207.


#### Create Map of Toronto with Neighborhoods

In [51]:
trono_map = folium.Map(location=[latitude, lngitude], zoom_start=10)
# add neighborhood points
for lat, lng, borough, neighborhood in zip(trono_df['Latitude'], trono_df['Longitude'],
                                           trono_df['Borough'], trono_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(trono_map)  
#
trono_map

#### Explore and segment the neighborhoods using Folium
Set the creds...

In [52]:
CLIENT_ID = '5LI4U0H5UDCHUWLX5ARE3TVBGUEBTWHVR1JAOYEBP5ZID0ES' # your Foursquare ID
CLIENT_SECRET = '43UBDAXUXQ52KLLDGF45GTX21BGCFE0LLNBY5NQMVCTDGF2A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5LI4U0H5UDCHUWLX5ARE3TVBGUEBTWHVR1JAOYEBP5ZID0ES
CLIENT_SECRET:43UBDAXUXQ52KLLDGF45GTX21BGCFE0LLNBY5NQMVCTDGF2A


Browse the first neighborhood

In [53]:
trono_df.loc[0, 'Neighborhood']

'Davisville'

Get long/lat

In [54]:
nb_lat = trono_df.loc[0, 'Latitude']
nb_lng = trono_df.loc[0, 'Longitude']
nb_name = trono_df.loc[0, 'Neighborhood'] # neighborhood name
print('long/lats of {} are {}, {}.'.format(nb_name, nb_lng, nb_lat))

long/lats of Davisville are -79.3887901, 43.7043244.


Get top 100 venues

In [55]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, nb_lat, nb_lng, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=5LI4U0H5UDCHUWLX5ARE3TVBGUEBTWHVR1JAOYEBP5ZID0ES&client_secret=43UBDAXUXQ52KLLDGF45GTX21BGCFE0LLNBY5NQMVCTDGF2A&v=20180605&ll=43.7043244,-79.3887901&radius=500&limit=100'

Get/review JSON data

In [106]:
jdat = requests.get(url).json()

Whoa! Lot's to look at. ``['response']['groups'][``idxval``]['items']`` has what we want. Extract it.

In [57]:
venues = jdat['response']['groups'][0]['items']

In [62]:
venues[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ae6ea6ef964a52082a721e3',
  'name': 'Jules Cafe Patisserie',
  'location': {'address': '617 Mt Pleasant Ave',
   'crossStreet': 'at Manor Rd E',
   'lat': 43.70413799694304,
   'lng': -79.38841260442167,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.70413799694304,
     'lng': -79.38841260442167}],
   'distance': 36,
   'postalCode': 'M4S 2M5',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['617 Mt Pleasant Ave (at Manor Rd E)',
    'Toronto ON M4S 2M5',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1d0941735',
    'name': 'Dessert Shop',
    'pluralName': 'Dessert Shops',
    'shortName': 'Desserts',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0,

In [63]:
len(venues)

37

Create function to get the category type

In [65]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Create DataFrame of the venues nearby

In [66]:
# nearby venues
df_venues_nearby = json_normalize(venues)
# filter columns
col_filt = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_venues_nearby = df_venues_nearby.loc[:, col_filt]
# filter categories
df_venues_nearby['venue.categories'] = df_venues_nearby.apply(get_category_type, axis=1)
# cleanup the columns
df_venues_nearby.columns = [col.split(".")[-1] for col in df_venues_nearby.columns]
# spot check the result
df_venues_nearby.head()

,name,categories,lat,lng
0,Jules Cafe Patisserie,Dessert Shop,43.704138,-79.388413
1,Thobors Boulangerie Patisserie Café,Café,43.704514,-79.388616
2,XO Gelato,Dessert Shop,43.705177,-79.388793
3,Marigold Indian Bistro,Indian Restaurant,43.702881,-79.388008
4,Viva Napoli,Pizza Place,43.705752,-79.389125


In [67]:
df_venues_nearby.shape

(37, 4)

## 2. Explore Toronto Neighborhoods

#### Function to automate neighborhood venue data collection

In [68]:
burl = 'https://api.foursquare.com/v2/venues/'
uri1 = 'explore?&client_id={}&client_secret={}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, LIMIT)
url1 = burl + uri1
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        #    CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        url = url1 + '&ll={},{}&radius={}'.format(lat, lng, radius)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                  'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

#### Run the function

In [71]:
l_names = trono_df['Neighborhood']
l_latitudes = trono_df['Latitude']
l_longitudes = trono_df['Longitude']
trono_venues = get_nearby_venues(l_names, l_latitudes, l_longitudes)

Davisville
Davisville North
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Forest Hill North, Forest Hill West
Lawrence Park
Moore Park, Summerhill East
North Toronto West
Roselawn
The Annex, North Midtown, Yorkville
Adelaide, King, Richmond
Berczy Park
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Cabbagetown, St. James Town
Central Bay Street
Chinatown, Grange Park, Kensington Market
Christie
Church and Wellesley
Commerce Court, Victoria Hotel
Design Exchange, Toronto Dominion Centre
First Canadian Place, Underground city
Harbord, University of Toronto
Harbourfront East, Toronto Islands, Union Station
Harbourfront, Regent Park
Rosedale
Ryerson, Garden District
St. James Town
Stn A PO Boxes 25 The Esplanade
Business reply mail Processing Centre969 Eastern
Studio District
The Beaches
The Beaches West, India Bazaar
The Danforth West, Riverdale
Brockton, Exhibition Place, Parkdale Village
Dovercourt Village, 

#### Check DataFrame size and venue counts

In [72]:
trono_venues.shape

(1707, 7)

In [73]:
trono_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Davisville,43.704324,-79.38879,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
1,Davisville,43.704324,-79.38879,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
2,Davisville,43.704324,-79.38879,XO Gelato,43.705177,-79.388793,Dessert Shop
3,Davisville,43.704324,-79.38879,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
4,Davisville,43.704324,-79.38879,Viva Napoli,43.705752,-79.389125,Pizza Place


Neighborhood venue counts

In [74]:
trono_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24
Business reply mail Processing Centre969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


#### Total unique categories

In [75]:
print('Unique categories: {}'.format(len(trono_venues['Venue Category'].unique())))

Unique categories: 232


## 3. Analyze Each Neighborhood

In [77]:
# code from the lab: one hot encoding
trono_onehot = pd.get_dummies(trono_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
trono_onehot['Neighborhood'] = trono_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [trono_onehot.columns[-1]] + list(trono_onehot.columns[:-1])
trono_onehot = trono_onehot[fixed_columns]

trono_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [78]:
trono_onehot.shape

(1707, 232)

Group neighborhoods with mean of categories...

In [79]:
trono_grouped = trono_onehot.groupby('Neighborhood').mean().reset_index()
trono_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.0100,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.00,0.020000,0.010000,0.00,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.020000,0.010000,0.070000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0100,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0000,0.00

In [80]:
trono_grouped.shape

(38, 232)

Print neighborhoods and their 5 top venues

In [81]:
num_top_venues = 5

for hood in trono_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = trono_grouped[trono_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.06
2    Restaurant  0.04
3    Steakhouse  0.04
4      Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
                   venue  freq
0            Coffee Shop  0.12
1                   Café  0.08
2  Performing Arts Venue  0.08
3              Nightclub  0.08
4         Breakfast Spot  0.08


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1       Burrito Place  0.05
2       Garden Center  0.05
3              Garden  0.05
4                 Spa  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1 

#### Create the pandas DataFrame...

Function to sort venues descending.

In [84]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

DataFrame to display top ten venues by neighborhood.

In [85]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:  # EAFP
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
trono_venues_sorted = pd.DataFrame(columns=columns)
trono_venues_sorted['Neighborhood'] = trono_grouped['Neighborhood']
for ind in np.arange(trono_grouped.shape[0]):
    trono_venues_sorted.iloc[ind, 1:] = most_common_venues(trono_grouped.iloc[ind, :], num_top_venues)

trono_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Restaurant,Gym,Hotel,Concert Hall
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Café,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Nightclub,Breakfast Spot,Café,Performing Arts Venue,Falafel Restaurant,Pet Store,Caribbean Restaurant,Italian Restaurant,Bar
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Butcher,Recording Studio,Burrito Place,Restaurant,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Airport Service,Airport Lounge,Airport Gate,Boutique,Plane,Sculpture Garden,Harbor / Marina,Airport Food Court,Airport
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Indian Restaurant,Bakery,Italian Restaurant,Pizza Place,Pub,Park,Café,Butcher
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Ice Cream Shop,Falafel Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Bakery,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Burger Joint
8,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Diner,Italian Restaurant,Restaurant,Coffee Shop,Convenience Store
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Pub,Café,Gastropub,Fast Food Restaurant


## 4. Cluster Neighborhoods

Use k-Means to make 5 clusters

In [86]:
kclusters = 5
trono_grouped_clustering = trono_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(trono_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

Create DataFrame with cluster and top ten venues in ``trono_venues_sorted``

In [87]:
trono_merged = trono_df  # manhattan_data
# add clustering labels
trono_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
trono_merged = trono_merged.join(trono_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

trono_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Davisville,43.704324,-79.388790,2,Dessert Shop,Sandwich Place,Italian Restaurant,Pizza Place,Seafood Restaurant,Café,Coffee Shop,Sushi Restaurant,Restaurant,Diner
1,Central Toronto,Davisville North,43.712751,-79.390197,2,Dance Studio,Burger Joint,Gym / Fitness Center,Park,Sandwich Place,Clothing Store,Breakfast Spot,Hotel,Food & Drink Shop,Women's Store
2,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,2,Coffee Shop,Pub,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Supermarket,Convenience Store,Pizza Place,Sushi Restaurant,Sports Bar
3,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,1,Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Dim Sum Restaurant,Park,Swim School,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Visualize the Clusters

In [89]:
### These variables were set above:
##cityname = 'Toronto, Canada'
##geoloc8r = Nominatim()
##location = geoloc8r.geocode(cityname)
##latitude = location.latitude
##lngitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, lngitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(trono_merged['Latitude'], trono_merged['Longitude'],
                                  trono_merged['Neighborhood'], trono_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [93]:
def get_cluster(idx):
    clust  = trono_merged.loc[trono_merged['Cluster Labels'] == idx, 
                          trono_merged.columns[[1] + list(range(5, trono_merged.shape[1]))]]
    return clust

In [100]:
l_clusters = []
for i in range(kclusters):
    l_clusters.append(get_cluster(i))

### Show DataFrame for Each Cluster

#### Cluster 1

In [101]:
l_clusters[0]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"Ryerson, Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bar,Japanese Restaurant,Sandwich Place,Ramen Restaurant,Plaza,Diner
27,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Butcher,Recording Studio,Burrito Place,Restaurant,Brewery


#### Cluster 2

In [102]:
l_clusters[1]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Forest Hill North, Forest Hill West",Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
12,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Indian Restaurant,Bakery,Italian Restaurant,Pizza Place,Pub,Park,Café,Butcher
22,"Harbourfront, Regent Park",Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,Shoe Store
36,"Parkdale, Roncesvalles",Breakfast Spot,Gift Shop,Movie Theater,Italian Restaurant,Dog Run,Burger Joint,Dessert Shop,Bar,Cuban Restaurant,Eastern European Restaurant


#### Cluster 3

In [103]:
l_clusters[2]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Italian Restaurant,Pizza Place,Seafood Restaurant,Café,Coffee Shop,Sushi Restaurant,Restaurant,Diner
1,Davisville North,Dance Studio,Burger Joint,Gym / Fitness Center,Park,Sandwich Place,Clothing Store,Breakfast Spot,Hotel,Food & Drink Shop,Women's Store
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Pub,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Supermarket,Convenience Store,Pizza Place,Sushi Restaurant,Sports Bar
4,Lawrence Park,Bus Line,Dim Sum Restaurant,Park,Swim School,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,"Moore Park, Summerhill East",Intersection,Restaurant,Park,Playground,Tennis Court,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
6,North Toronto West,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner,Mexican Restaurant,Salon / Barbershop
7,Roselawn,Garden,Pool,Home Service,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
8,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Coffee Shop,Pizza Place,Park,American Restaurant,Liquor Store,Burger Joint,Jewish Restaurant,BBQ Joint
9,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Restaurant,Gym,Hotel,Concert Hall
10,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Café,Bakery


#### Cluster 4

In [104]:
l_clusters[3]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Gym,Steakhouse,Deli / Bodega


#### Cluster 5

In [105]:
l_clusters[4]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Cheese Shop,Fish Market,Juice Bar,Latin American Restaurant
